# 5.2. Deep Learning Methods

Neural networks, especially autoencoders, can be effective in imputing missing values in complex datasets. Deep learning methods, particularly neural networks like autoencoders, offer a powerful approach for imputing missing values in complex datasets. These methods are especially useful when the data has intricate, non-linear relationships that traditional statistical methods might not capture effectively.

**Understanding Autoencoders for Imputation:**

**1) What is an Autoencoder?**

-An autoencoder is a type of neural network that is trained to copy its input to its output.

-It has a hidden layer that describes a code used to represent the input.

-The network may be viewed as consisting of two parts: an encoder function, which compresses the input into a latent-space representation, and a decoder function, which reconstructs the input from the latent space.

**2) How Autoencoders Work for Imputation:**

-The key idea is to train the autoencoder to ignore the noise (missing values) in the input data.

-During training, inputs with missing values are presented, and the network learns to predict the missing values in a way that minimizes reconstruction error for known parts of the data.

-This results in the network learning a robust representation
of the data, enabling it to make reasonable guesses about missing values.

**3) Advantages of Using Autoencoders:**

**~Handling Complex Patterns:~** They can capture non-linear relationships in the data, which is particularly useful for complex datasets.

**~Scalability:~ **They can handle large-scale datasets efficiently.

**~Flexibility~:** They can be adapted to different types of data (e.g., images, text, time-series).

**4) Implementation Considerations:**

**Data Preprocessing**: Data should be normalized or standardized before feeding it into an autoencoder.

**Network Architecture:** The choice of architecture (number of layers, type of layers, etc.) depends on the complexity of the data.

**Training Process:** It might involve techniques like dropout or noise addition to improve the model's ability to handle missing data.

**Example Use-Cases:**

**Image Data:** Filling in missing pixels or reconstructing corrupted images.

**Time-Series Data:** Imputing missing values in sequences like stock prices or weather data.

**Tabular Data:** Handling missing entries in datasets used for machine learning.

# Implementation Example:

Here's a simplified example of how you might set up an autoencoder for imputation in Python using TensorFlow and Keras: (Check the next notebook)

In [1]:
!pip install tensorflow

  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.5-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached wrapt-2.1.1-cp311-cp311-win_amd64.whl.metadata (7.6 kB)
  Using cached grpcio-1.78.1-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached 

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\choice laptops\\AppData\\Local\\Temp\\pip-unpack-2e_fcbvk\\tensorflow-2.20.0-cp311-cp311-win_amd64.whl'
Consider using the `--user` option or check the permissions.



In [ ]:
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the Titanic dataset
df_titanic = sns.load_dataset('titanic')

# Selecting relevant features for simplicity
df_titanic = df_titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

# Preprocessing
# Separate features and target
X = df_titanic.drop('survived', axis=1)
y = df_titanic['survived']

# Handling missing values and categorical variables
numeric_features = ['age', 'fare', 'sibsp', 'parch']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())])

categorical_features = ['pclass', 'sex', 'embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Preprocessing the dataset
X_preprocessed = preprocessor.fit_transform(X)

# Splitting the dataset (we'll use the train set to train the autoencoder)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Using the autoencoder for imputation on test set
X_test_imputed = autoencoder.predict(X_test)

# Note: Transforming imputed data back to original feature space is complex and requires reversing the preprocessing steps. 
# This is often not straightforward, especially for one-hot encoded features.

ModuleNotFoundError: No module named 'tensorflow'